Context
Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

The original MNIST dataset contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. "If it doesn't work on MNIST, it won't work at all", they said. "Well, if it does work on MNIST, it may still fail on others."

Zalando seeks to replace the original MNIST dataset

Content
- Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

- To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix. For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.


Labels --->

0. T-shirt/top
1. Trouser
2. Pullover
3. Dress
4. Coat
5. Sandal
6. Shirt
7. Sneaker
8. Bag
9. Ankle boot


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical

In [2]:
train = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

In [3]:
print('Training Data:', train.shape)
print('Test Data:', test.shape)

Training Data: (60000, 785)
Test Data: (10000, 785)


In [4]:
train_X = np.array(train.drop('label',axis=1))
train_Y = np.array(train['label'])
test_X = np.array(test.drop('label',axis=1))
test_Y = np.array(test['label'])

In [5]:
train_X.shape,train_Y.shape,test_X.shape,test_Y.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [6]:
classes = np.unique(train_Y)
nclasses = len(classes)
print('Total number of classes:', nclasses)
print('Output classes:', classes)

Total number of classes: 10
Output classes: [0 1 2 3 4 5 6 7 8 9]


In [7]:
# Data Preprocessing
train_X = train_X.reshape(-1,28,28,1)
test_X = test_X.reshape(-1,28,28,1)
train_X.shape,test_X.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [8]:
# Changing datatype
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
# Rescaling the pixel values
train_X = train_X/255.0
test_X = test_X/255.0

In [9]:
# Changing the labels from categorical to one hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

Original label: 2
After conversion to one-hot: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
# Splitting the data
from sklearn.model_selection import train_test_split
train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [11]:
# Importing libraries for Modelling the Data
import keras
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [12]:
batch_size = 64
epochs = 20
num_classes = 10

In [13]:
fashion_model = Sequential()

fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(28,28,1)))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [14]:
fashion_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

In [15]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [16]:
fashion_train_dropout = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

Epoch 1/20
750/750 [==============================] - 4s 5ms/step - loss: 0.6085 - accuracy: 0.7722 - val_loss: 0.3707 - val_accuracy: 0.8621
Epoch 2/20
750/750 [==============================] - 3s 5ms/step - loss: 0.3857 - accuracy: 0.8605 - val_loss: 0.3171 - val_accuracy: 0.8837
Epoch 3/20
750/750 [==============================] - 5s 6ms/step - loss: 0.3355 - accuracy: 0.8774 - val_loss: 0.2916 - val_accuracy: 0.8917
Epoch 4/20
750/750 [==============================] - 4s 5ms/step - loss: 0.3086 - accuracy: 0.8867 - val_loss: 0.2648 - val_accuracy: 0.9017
Epoch 5/20
750/750 [==============================] - 3s 5ms/step - loss: 0.2863 - accuracy: 0.8943 - val_loss: 0.2605 - val_accuracy: 0.9019
Epoch 6/20
750/750 [==============================] - 4s 5ms/step - loss: 0.2733 - accuracy: 0.8989 - val_loss: 0.2515 - val_accuracy: 0.9066
Epoch 7/20
750/750 [==============================] - 3s 4ms/step - loss: 0.2603 - accuracy: 0.9037 - val_loss: 0.2377 - val_accuracy: 0.9115
Epoch 

In [17]:
# Model Evaluation on Test Data
test_eval = fashion_model.evaluate(test_X, test_Y_one_hot, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 0.2052 - accuracy: 0.9292
